<a href="https://colab.research.google.com/github/ElahehJafarigol/Federated-Learning-GANs/blob/main/Wassetstein_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Deep Imbalanced Learning for Weather Data: A Federated Learning Approach**

**The high-level algorithm for the Conditional Generative Adversarial Network (CGAN) is as follows:**

1. Load and preprocess the dataset
2. Define the generator and discriminator models
3. Define the CGAN model by combining the generator and discriminator models
4. Train the discriminator by feeding it real and generated data while adjusting its weights to improve classification accuracy
5. Train the generator by generating synthetic data and feeding it into the discriminator while adjusting its weights to maximize the probability of the generated data being classified as real
6. Repeat steps 4 and 5 for a fixed number of epochs
7. Save the trained generator model
8. Use the generator to generate synthetic data
9. Combine the synthetic data with the real data to create a balanced dataset
10. Train a classification model on the balanced dataset to evaluate its performance.

In [ ]:
!pip install --quiet keras==2.9.0
!pip install --quiet tensorflow==2.9.2

In [ ]:
!pip install --quiet imbalanced-learn

In [ ]:
import os
from  IPython import display
import pathlib
import shutil
import tempfile
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')
from pandas.core.common import random_state
import pandas as pd
import numpy as np
import random
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing

random_state = 42
random_seed = 42
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import SGD
from keras import backend as K
from keras import datasets, layers, models, metrics
from keras.layers import GaussianNoise
from keras import regularizers

from keras.layers.serialization import activation
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.layers import Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.activation import ReLU
from pandas.core.indexes.datetimes import Resolution
from keras.layers import Concatenate

import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.metrics import roc_auc_score, confusion_matrix

from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.backend import mean

####**Multiple Datasets**

###Load and preprocess the data

In [ ]:
# Uploading the data in Google colab
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

1. It imports the weather dataset as a pandas dataframe and preprocesses it. It drops columns that are not useful for the model and replaces the binary labels "RainToday" and "RainTomorrow" with 1 for "Yes" and 0 for "No." It also fills in missing data with the mean of the respective feature.

2. It separates the features and the labels into X and y, respectively, and normalizes the features using MinMaxScaler.

In [ ]:
# Import the data file as a dataframe
import io
weather_df = pd.read_csv(io.BytesIO(uploaded['WeatherAUS.csv']))
# Dataset is now stored in a Pandas Dataframe

weather_df.shape

Data = weather_df
Data.RainToday = [1 if each=="Yes" else 0 for each in Data.RainToday]
Data.RainTomorrow = [1 if each=="Yes" else 0 for each in Data.RainTomorrow]
#Data.head()

Data = Data.drop(['Sunshine','Evaporation','Cloud3pm','Cloud9am','RISK_MM','Location','Date','WindGustDir',
       'WindDir9am', 'WindDir3pm'],axis=1)
Data.shape

# replace rest of the nulls with respective means
fill_feat = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm',
             'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm',
             'RainToday', 'RainTomorrow']
for i in fill_feat:
    Data[i].fillna(np.mean(Data[i]),inplace=True)

Data.shape

Data.dropna(inplace=True)

# Separate the features and labels
X = Data.drop('RainTomorrow', axis=1)
y = Data['RainTomorrow']

print("X:", X.shape)
print("y:", y.shape)

# Normalize the features
scalar = preprocessing.MinMaxScaler(feature_range=(0, 12))
norm_data = scalar.fit_transform(X)
X = pd.DataFrame(norm_data, columns=[X.columns])
X = pd.DataFrame(X.reset_index(drop=True))
X.shape

X_complete = X
y_complete = y

X: (142193, 13)
y: (142193,)


In [ ]:
X = X_complete
y = y_complete

In [ ]:
latent_dim = 13
n_critic = 1
"""n_critic is a hyperparameter in the Wasserstein GAN (WGAN) training process
 that represents the number of times the discriminator is updated for each
 generator update. In the original WGAN paper, the authors recommend setting
 n_critic to 5, which means that for each iteration of the training loop,
 the discriminator is updated five times before the generator is updated once.
 This helps to prevent the generator from outpacing the discriminator and
 producing poor-quality samples."""

clip_value = 0.001
lambda_reg = 10

# Define the generator model
def define_generator(latent_dim):
    model = Sequential()
    model.add(Dense(1024, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))

    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))

    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))

    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(X.shape[1], activation='sigmoid'))

    # model.summary()
    return model

# Define the discriminator model
def define_discriminator(input_shape):
    model = Sequential()
    model.add(Dense(512, input_dim=input_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))

    # model.summary()
    return model

# Define the Wasserstein loss
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

# Train the WGANs model
def train(X, y, epochs, batch_size, latent_dim):

    # Remove rows with missing data
    X = X.dropna()

    # Get the set of valid indices
    valid_indices = set(X.index)

    # Define the WGANs model

    # Instantiate the generator
    generator = define_generator(latent_dim)
    discriminator = define_discriminator(X.shape[1])

    # Compile the discriminator model
    discriminator.compile(loss=wasserstein_loss,
                          optimizer=keras.optimizers.SGD(0.0003, 0.5),
                          metrics=['accuracy'])

    # Freeze the discriminator weights
    discriminator.trainable = False
    # Build the combined model (generator and discriminator)
    z = Input(shape=(latent_dim,))
    generated_data = generator(z)
    validity = discriminator(generated_data)
    combined_model = Model(z, validity)

    # Define the Wasserstein distance function
    def wasserstein_distance(y_true, y_pred):
        return K.mean(y_true) - K.mean(y_pred)

    # Add regularization to the generator loss
    def generator_loss(y_true, y_pred):
        return -K.mean(y_pred) + lambda_reg * K.mean(wasserstein_distance(y_true, y_pred))

    # Compile the combined model with the modified generator loss
    combined_model.compile(loss=generator_loss,
                           optimizer=keras.optimizers.SGD(0.0003, 0.5))

    # Train the discriminator
    for epoch in range(epochs):
        for _ in range(n_critic):
            try:
                # Sample a batch of real data
                real_indices = np.random.choice(list(valid_indices), size=batch_size, replace=False)
                real_data = X.loc[real_indices]

                # Sample a batch of noise
                noise = np.random.normal(0, 1, (batch_size, latent_dim))

                # Generate a batch of fake data
                generated_data = generator.predict(noise)

                # Combine real and fake data
                combined_data = np.concatenate([real_data, generated_data])

                # Create labels for real and fake data
                labels = np.concatenate([np.ones((batch_size, 1)), -np.ones((batch_size, 1))])

                # Train the discriminator
                d_loss = discriminator.train_on_batch(combined_data, labels)

                # Clip the discriminator weights
                for layer in discriminator.layers:
                    weights = layer.get_weights()
                    weights = [np.clip(w, -clip_value, clip_value) for w in weights]
                    layer.set_weights(weights)
            except KeyError as e:
                print(f"Skipping row {id} due to KeyError: {e}")
                continue

            # Train the generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            misleading_targets = np.ones((batch_size, 1))
            g_loss = combined_model.train_on_batch(noise, misleading_targets)

            # Print the progress
            print(f"Epoch {epoch} Discriminator Loss: {d_loss} Generator Loss: {g_loss}")


In [ ]:
train(X, y, epochs = 20, batch_size = 64, latent_dim = 13)

2/2 [==============================] - 0s 8ms/step
Epoch 0 Discriminator Loss: [0.09628389775753021, 0.40625] Generator Loss: 4.499964237213135
2/2 [==============================] - 0s 6ms/step
Epoch 1 Discriminator Loss: [-1.2759119272232056e-07, 0.5] Generator Loss: 4.4999470710754395
2/2 [==============================] - 0s 6ms/step
Epoch 2 Discriminator Loss: [-1.6763806343078613e-08, 0.5] Generator Loss: 4.499938011169434
2/2 [==============================] - 0s 6ms/step
Epoch 3 Discriminator Loss: [-8.009374141693115e-08, 0.5] Generator Loss: 4.499932765960693
2/2 [==============================] - 0s 7ms/step
Epoch 4 Discriminator Loss: [-1.6065314412117004e-07, 0.5] Generator Loss: 4.499931812286377
2/2 [==============================] - 0s 9ms/step
Epoch 5 Discriminator Loss: [-1.6298145055770874e-07, 0.5] Generator Loss: 4.4999308586120605
2/2 [==============================] - 0s 6ms/step
Epoch 6 Discriminator Loss: [-1.825392246246338e-07, 0.5] Generator Loss: 4.49992990

In [ ]:
original_indices = set(Data.index)
current_indices = set(X.index)

missing_indices = original_indices - current_indices
print(missing_indices)

set()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

# Calculate the class ratio in the balanced dataset
class_ratio = sum(y_train == 0) / len(y_train)
class_ratio

0.7755155862650984

In [ ]:
# Save the generator
generator = define_generator(latent_dim)

# Use the generator to create synthetic data
synthetic_data = generator.predict(np.random.normal(0, 1, (len(X_train), latent_dim)))

# Combine the real and synthetic data to create a balanced dataset
X_balanced = np.concatenate([X_train, synthetic_data])
y_balanced = np.concatenate([y_train, np.zeros(len(X_train))])

# Calculate the class ratio in the balanced dataset
class_ratio = sum(y_balanced == 0) / len(y_balanced)
print(class_ratio)

print(X_balanced.shape)
print(X_train.shape)

3555/3555 [==============================] - 11s 3ms/step
0.8877577931325492
(227508, 13)
(113754, 13)


#####Training Federated Learning

In [ ]:
# define data shape and number of classes
input_shape = (13,)
num_classes = 2
num_test_samples = len(X_test)
num_train_samples = len(X_train)
Batch_size = 64
communication_round = 10
Learning_rate = 0.001
Momentum = 0.9
Epochs = 30
data_shape = (13,)

In [ ]:
def create_clients(X_train, y_train, num_clients, initial='client'):
    # create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    # randomize the data before splitting the data between clients
    indices = np.random.permutation(len(X_train))
    X_shuffled = X_train[indices]
    y_shuffled = y_train[indices]

    # shard data and place at each client
    size = len(X_train) // num_clients
    remainder = len(X_train) % num_clients
    shards = [(X_shuffled[i:i+size], y_shuffled[i:i+size]) for i in range(0, size*num_clients, size)]
    if remainder:
        shards[-1] = (np.concatenate([shards[-1][0], X_shuffled[-remainder:]]),
                      np.concatenate([shards[-1][1], y_shuffled[-remainder:]]))

    # number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    # create dictionary of clients and their shards
    clients = {}
    for i in range(len(client_names)):
        # create dictionary entry with client name and data shape
        clients[client_names[i]] = (shards[i][0].shape, shards[i][1].shape)

    return clients

def batch_data(data_shard, batchsize=Batch_size):
    # unpack data shard into data and labels arrays
    X, y = data_shard

    # create a tensorflow dataset object from the data and labels
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    # shuffle and batch the dataset
    return dataset.shuffle(len(y)).batch(batchsize)

# create clients from training data and associated labels
X_train = X_balanced
y_train = y_balanced
clients = create_clients(X_train, y_train, num_clients=10, initial='client')
for client in clients:
    print(f"{client}: X={clients[client][0]}, y={clients[client][1]}")

# You can access the shards for each client by indexing the dictionary returned
# by create_clients(), like so:
client_1_shard = clients['client_1']
client_1_shard

# create a batched dataset for each client's data shard
clients_batched = {}
for client in clients:
    client_data_shard = (X_train, y_train)
    batched_data = batch_data(client_data_shard)
    clients_batched[client] = batched_data

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

# Federated Averaging: aggregation method
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the batch_size
    batch_size = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*batch_size
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*batch_size
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    # Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

def test_model(X_test, Y_test, model, communication_round):
    cce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    logits = model.predict(X_test, batch_size=Batch_size)
    logits = tf.squeeze(logits, axis=1) # remove last dimension from logits
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.round(logits), Y_test)
    auc = roc_auc_score(Y_test, logits)
    tn, fp, fn, tp = confusion_matrix(Y_test, tf.round(logits)).ravel()
    g_mean = np.sqrt(tp/(tp+fn)*tn/(tn+fp))
    print('communication_round: {} | global_accuracy: {:.3%} | global_loss: {} | global_AUC: {:.3%} | global_G-mean: {:.3%}'.format(communication_round, acc, loss, auc, g_mean))
    return acc, loss, auc, g_mean

client_1: X=(22750, 13), y=(22750,)
client_2: X=(22750, 13), y=(22750,)
client_3: X=(22750, 13), y=(22750,)
client_4: X=(22750, 13), y=(22750,)
client_5: X=(22750, 13), y=(22750,)
client_6: X=(22750, 13), y=(22750,)
client_7: X=(22750, 13), y=(22750,)
client_8: X=(22750, 13), y=(22750,)
client_9: X=(22750, 13), y=(22750,)
client_10: X=(22758, 13), y=(22758,)


In [ ]:
epochs = Epochs
learning_rate = Learning_rate
momentum = Momentum
batch_size = Batch_size

# Define the optimizer and loss function
optimizer = keras.optimizers.SGD(learning_rate=0.01,
                                 momentum = momentum,
                                 nesterov = False)
loss = "binary_crossentropy"

metrics = [keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      ]

class MyModel:
    @staticmethod
    def build(shape, classes):
        model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(13,)),
        layers.GaussianNoise(stddev = 0.5),
        layers.Dense(128, activation='relu'),
        layers.GaussianNoise(stddev = 0.3),
        layers.Dense(64, activation='relu'),
        layers.GaussianNoise(stddev = 0.1),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
        ])
        return model

In [ ]:
smlp_global = MyModel()
global_model = smlp_global.build(shape=(13,), classes=2)
global_model.summary()

global_acc_list = []
global_loss_list = []
global_auc_list = []
global_gmean_list = []

# commence global training loop
for communication_round in range(communication_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    # initial list to collect local model weights after scaling
    scaled_local_weight_list = list()

    # randomize client data - using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)

    # loop through each client and create new local model
    for client in client_names:
        smlp_local = MyModel()
        local_model = smlp_local.build(shape=(13,), classes=2)
        #local_model = smlp_local.build()
        local_model.compile(loss=loss,
                            optimizer=optimizer,
                            metrics=metrics)

        # set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        # fit local model with client's data
        model_history = local_model.fit(clients_batched[client], epochs=epochs, verbose=1)

        # evaluate local model on test data
        # test_loss, test_accuracy = local_model.evaluate(X_test, y_test)
        # print(f'Local model {client} - test_loss: {test_loss} - test_accuracy: {test_accuracy}')

        # scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        # clear session to free memory after each communication round
        K.clear_session()

    # to get the average over all the local model, we simply take the sum of the scaled weights
    avg_grad = sum_scaled_weights(scaled_local_weight_list)

    # update global model
    global_model.set_weights(avg_grad)

    # test global model and print out metrics after each communications round
    for X_test, y_test in test_batched:
        global_acc, global_loss, global_auc, global_gmean = test_model(X_test, y_test, global_model, communication_round)
        global_acc_list.append(global_acc)
        global_loss_list.append(global_loss)
        global_auc_list.append(global_auc)
        global_gmean_list.append(global_gmean)

Streaming output truncated to the last 5000 lines.
Epoch 30/30
3555/3555 [==============================] - 15s 4ms/step - loss: 0.1791 - tp: 12401.0000 - fp: 4319.0000 - tn: 197653.0000 - fn: 13135.0000 - accuracy: 0.9233 - precision: 0.7417 - recall: 0.4856 - auc: 0.9403 - prc: 0.7002
Epoch 1/30
3555/3555 [==============================] - 18s 4ms/step - loss: 0.1821 - tp: 24421.0000 - fp: 8657.0000 - tn: 395287.0000 - fn: 26651.0000 - accuracy: 0.9224 - precision: 0.7383 - recall: 0.4782 - auc: 0.9392 - prc: 0.6954
Epoch 2/30
3555/3555 [==============================] - 15s 4ms/step - loss: 0.1814 - tp: 12200.0000 - fp: 4320.0000 - tn: 197652.0000 - fn: 13336.0000 - accuracy: 0.9224 - precision: 0.7385 - recall: 0.4778 - auc: 0.9385 - prc: 0.6926
Epoch 3/30
3555/3555 [==============================] - 15s 4ms/step - loss: 0.1812 - tp: 12275.0000 - fp: 4362.0000 - tn: 197610.0000 - fn: 13261.0000 - accuracy: 0.9225 - precision: 0.7378 - recall: 0.4807 - auc: 0.9386 - prc: 0.6942
Epoc

####**Single Dataset**

###Load and preprocess the data

In [ ]:
# Uploading the data in Google colab
from google.colab import files

uploaded_1 = files.upload()

for fn in uploaded_1.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded_1[fn])))

Saving Client1.csv to Client1.csv
User uploaded file "Client1.csv" with length 2266695 bytes


In [ ]:
# Import the data file as a dataframe
import io
weather_df = pd.read_csv(io.BytesIO(uploaded_1['Client1.csv']))
# Dataset is now stored in a Pandas Dataframe

weather_df.shape

(22750, 24)

In [ ]:
Data = weather_df
Data.RainToday = [1 if each=="Yes" else 0 for each in Data.RainToday]
Data.RainTomorrow = [1 if each=="Yes" else 0 for each in Data.RainTomorrow]
#Data.head()

Data = Data.drop(['Sunshine','Evaporation','Cloud3pm','Cloud9am','RISK_MM','Location','Date','WindGustDir',
       'WindDir9am', 'WindDir3pm'],axis=1)
Data.shape

# replace rest of the nulls with respective means
fill_feat = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm',
             'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm',
             'RainToday', 'RainTomorrow']
for i in fill_feat:
    Data[i].fillna(np.mean(Data[i]),inplace=True)

Data.shape
Data.dropna(inplace=True)

# Separate the features and labels
X = Data.drop('RainTomorrow', axis=1)
y = Data['RainTomorrow']

print("X:", X.shape)
print("y:", y.shape)

# Normalize the features
scalar = preprocessing.MinMaxScaler(feature_range=(0, 12))
norm_data = scalar.fit_transform(X)
X = pd.DataFrame(norm_data, columns=[X.columns])
X = pd.DataFrame(X.reset_index(drop=True))

original_indices = set(Data.index)
current_indices = set(X.index)

missing_indices = original_indices - current_indices
print(missing_indices)

X_single = X
y_single = y

counter = Counter(y_single)
print(counter)

X: (22750, 13)
y: (22750,)
set()
Counter({0: 17598, 1: 5152})


#####Data Augmentation

In [ ]:
train(X, y, epochs = 20, batch_size = 128, latent_dim = 13)

# Save the generator
generator_0 = define_generator(latent_dim)

4/4 [==============================] - 1s 6ms/step
Epoch 0 Discriminator Loss: [0.043468885123729706, 0.3359375] Generator Loss: 4.499978065490723
4/4 [==============================] - 0s 5ms/step
Epoch 1 Discriminator Loss: [-1.1920928955078125e-07, 0.484375] Generator Loss: 4.499966621398926
4/4 [==============================] - 0s 6ms/step
Epoch 2 Discriminator Loss: [-2.0978040993213654e-07, 0.49609375] Generator Loss: 4.499960899353027
4/4 [==============================] - 0s 6ms/step
Epoch 3 Discriminator Loss: [-9.150244295597076e-08, 0.5] Generator Loss: 4.499958038330078
4/4 [==============================] - 0s 7ms/step
Epoch 4 Discriminator Loss: [-2.1909363567829132e-07, 0.5] Generator Loss: 4.499957084655762
4/4 [==============================] - 0s 7ms/step
Epoch 5 Discriminator Loss: [-1.1431984603404999e-07, 0.5] Generator Loss: 4.499955177307129
4/4 [==============================] - 0s 6ms/step
Epoch 6 Discriminator Loss: [-1.6880221664905548e-07, 0.5] Generator Lo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

# Calculate the class ratio in the balanced dataset
class_ratio = sum(y_train == 0) / len(y_train)
class_ratio

0.7730769230769231

In [ ]:
# Use the generator to create synthetic data
synthetic_data = generator_0.predict(np.random.normal(0, 1, (len(X_train), latent_dim)))

# Combine the real and synthetic data to create a balanced dataset
X_balanced = np.concatenate([X_train, synthetic_data])
y_balanced = np.concatenate([y_train, np.zeros(len(X_train))])

print("X_train", X_train.shape)
print("X_balanced", X_balanced.shape)

print("X_balanced:", X_balanced.shape)
print("y_balanced:", y_balanced.shape)

569/569 [==============================] - 3s 5ms/step
X_train (18200, 13)
X_balanced (36400, 13)
X_balanced: (36400, 13)
y_balanced: (36400,)


#####Centralized Training

In [ ]:
# define data shape and number of classes
input_shape = (13,)
num_classes = 2
num_test_samples = len(X_test)
num_train_samples = len(X_train)
Batch_size = 64
communication_round = 10
Learning_rate = 0.001
Momentum = 0.9
Epochs = 30
data_shape = (13,)

In [ ]:
epochs = Epochs
learning_rate = Learning_rate
momentum = Momentum
batch_size = Batch_size

# Define the optimizer and loss function
optimizer = keras.optimizers.SGD(learning_rate=0.01,
                                 momentum = momentum,
                                 nesterov = False)
loss = "binary_crossentropy"

metrics = [keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      ]

model = tf.keras.Sequential([
layers.Dense(128, activation='relu', input_shape=(13,)),
layers.Dense(64, activation='relu'),
layers.Dense(32, activation='relu'),
layers.Dense(1, activation='sigmoid')
])


model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

model.fit(X_balanced, y_balanced, epochs=Epochs, batch_size=Batch_size)

Epoch 1/30
569/569 [==============================] - 6s 2ms/step - loss: 0.2265 - tp: 1773.0000 - fp: 917.0000 - tn: 31353.0000 - fn: 2357.0000 - accuracy: 0.9101 - precision: 0.6591 - recall: 0.4293 - auc: 0.9001 - prc: 0.5732
Epoch 2/30
569/569 [==============================] - 1s 2ms/step - loss: 0.1927 - tp: 1927.0000 - fp: 818.0000 - tn: 31452.0000 - fn: 2203.0000 - accuracy: 0.9170 - precision: 0.7020 - recall: 0.4666 - auc: 0.9309 - prc: 0.6603
Epoch 3/30
569/569 [==============================] - 1s 2ms/step - loss: 0.1907 - tp: 1951.0000 - fp: 782.0000 - tn: 31488.0000 - fn: 2179.0000 - accuracy: 0.9187 - precision: 0.7139 - recall: 0.4724 - auc: 0.9325 - prc: 0.6635
Epoch 4/30
569/569 [==============================] - 1s 2ms/step - loss: 0.1891 - tp: 1933.0000 - fp: 797.0000 - tn: 31473.0000 - fn: 2197.0000 - accuracy: 0.9177 - precision: 0.7081 - recall: 0.4680 - auc: 0.9337 - prc: 0.6700
Epoch 5/30
569/569 [==============================] - 1s 3ms/step - loss: 0.1881 - t

In [ ]:
Evaluation = model.evaluate(X_test, y_test, verbose = 1)

143/143 [==============================] - 3s 2ms/step - loss: 0.3672 - tp: 474.0000 - fp: 177.0000 - tn: 3351.0000 - fn: 548.0000 - accuracy: 0.8407 - precision: 0.7281 - recall: 0.4638 - auc: 0.8565 - prc: 0.6883
